<a href="https://colab.research.google.com/github/fwangliberty/AIoTDesign-Frontend/blob/master/IoT_Security_Use_Case2_Overall_IDS_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 7: IoT Security

# Use Case 2: Network Intrusion Detection

# Model: Simple DNN

# Step 1: Data Collection (downloading)

For this use case we are using famous open source KDDCUP99 data. To speed up the pocess, we have added a pre-processed dataset in the use case 2 folder.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Training.csv to Training.csv


In [2]:
from google.colab import files
uploaded = files.upload()

Saving Testing.csv to Testing.csv


# Step 2: Pre-processing

# Step 3: Model Training and Validation

# Data loading and preparation for Training

In [3]:
from __future__ import print_function
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score)
from sklearn.preprocessing import Normalizer
#import h5py
from keras import callbacks
from keras.callbacks import CSVLogger
from keras.callbacks import TensorBoard
from sklearn.metrics import confusion_matrix

# Load the pre-processed training and test data
traindata = pd.read_csv('Training.csv', header=None)
testdata = pd.read_csv('Testing.csv', header=None)

## For OMP error
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,0]
C = testdata.iloc[:,0]
T = testdata.iloc[:,1:42]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)


X_train = np.array(trainX)
X_test = np.array(testT)

batch_size = 64


# Model Generation and Training

In [5]:
# Generate or define the netwrok
model = Sequential()
model.add(Dense(1024,input_dim=41,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(512,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(256,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# Use tensorboad to log files
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

checkpointer = callbacks.ModelCheckpoint(filepath="checkpoint-{epoch:02d}.hdf5", 
                                         verbose=1, save_best_only=True, monitor='loss')
csv_logger = CSVLogger('training_set_dnnanalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, batch_size=batch_size, epochs=1, callbacks=[checkpointer,csv_logger, tensorboard])
model.save("dnn_model.hdf5")


7720/7720 [==============================] - 180s 23ms/step - loss: 0.0210 - accuracy: 0.9925

Epoch 00001: loss improved from inf to 0.01336, saving model to checkpoint-01.hdf5


# Performance Evaluation/Validation

In [6]:
# Validation & Confusion Matrix
score = []
name = []

for file in os.listdir('.'):
  model.load_weights("checkpoint-01.hdf5")
  y_train1 = y_test
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_train1, y_pred)
  recall = recall_score(y_train1, y_pred , average="binary")
  precision = precision_score(y_train1, y_pred , average="binary")
  f1 = f1_score(y_train1, y_pred, average="binary")
  print("----------------------------------------------")
  print("accuracy")
  print("%.3f" %accuracy)
  score.append(accuracy)
  name.append(file)


model.load_weights("./"+name[score.index(max(score))])
pred = model.predict(X_test)
proba = model.predict_proba(X_test)
np.savetxt("dnn4predicted.txt", pred)
np.savetxt("dnn4probability.txt", proba)

accuracy = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred , average="binary")
precision = precision_score(y_test, pred , average="binary")
f1 = f1_score(y_test, pred, average="binary")


print("----------------------------------------------")
print("accuracy")
print("%.3f" %accuracy)

### Confusion Matrix 
import matplotlib.pylab as plt
# This is to show plot 
%matplotlib inline
matrix = confusion_matrix(y_test, y_pred)
print (matrix)

#Plotting the Confusion Matrix
labels = ['Normal', 'Intruder']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(matrix)
#plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


----------------------------------------------
accuracy
0.922
----------------------------------------------
accuracy
0.922
----------------------------------------------
accuracy
0.922
----------------------------------------------
accuracy
0.922
----------------------------------------------
accuracy
0.922
----------------------------------------------
accuracy
0.922
----------------------------------------------
accuracy
0.922
----------------------------------------------
accuracy
0.922


OSError: ignored

In [7]:
!ls

checkpoint-01.hdf5  logs	 Testing.csv   training_set_dnnanalysis.csv
dnn_model.hdf5	    sample_data  Training.csv
